In [1]:
from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from morsePotential import morsePotential
from notebookImporter import importNotebook

HartreeFock = importNotebook("Hartree_Class")

#set up the automated calculations used to generate the surface potential

#User Specified Variables

#Enter the atomic number for atoms 1 and 2
atom1 = 1
atom2 = 1

#Enter the number of electrons for atom 1 and 2
#Note the total number of electrons must be a multiple of 2, 
#otherwise, the computation will round the total number of electrons 
#down to the nearest multiple of 2
atomN1 = 1
atomN2 = 1

#Select a basis, for a list of available basis sets, please see the "basisSets" folder
basisName = "DEF2-SVP"

#Enter Starting and End Bond distance for the diatomic molecule
startR = 0.4
endR = 3.5

#Please enter the distance the bond distance should increase each iteration
#The smaller the number, the more accurate and time-consuming the computation will be
step = 0.1

#Please enter how small the difference in energy between two ground state energy calculations
#must be in order to convergence to occur
convergenceCritera = pow(10, -10)

#Please enter the maximum number of iterations the SCF procedure can take before
#the computation will fail as being unable to converge
maxIterations = 100

#System Defined Variables
E = []
R = []
MOEnergy = []
delta = (endR - startR) / step
currentR = startR

while(not(currentR >= endR)):
    
    #define the molecular system
    system = molecule()
    system.addAtom(atom(vector(1,1,2+currentR), atom1, atomN1))
    system.addAtom(atom(vector(1,1,2), atom2, atomN2))

    system.addBasis(basisName)
    
    #Set up the Hartree Procedure
    HF = HartreeFock.HF(system, convergenceCritera, maxIterations)

    #Store Bond distance, associated Energy, and MO Energies
    R.append(currentR)
    E.append(HF.SCF())
    MOEnergy.append(HF.MOEnergy)
    
    print("Hartree-Fock Computation: " + str(currentR * 100 // endR) + "%")
    
    #update the current intenuclear distance
    currentR += step

print("Hartree-Fock Computation Complete")

Hartree-Fock Computation: 11.0%
Hartree-Fock Computation: 14.0%
Hartree-Fock Computation: 17.0%
Hartree-Fock Computation: 20.0%
Hartree-Fock Computation: 22.0%
Hartree-Fock Computation: 25.0%
Hartree-Fock Computation: 28.0%
Hartree-Fock Computation: 31.0%
Hartree-Fock Computation: 34.0%
Hartree-Fock Computation: 37.0%
Hartree-Fock Computation: 40.0%
Hartree-Fock Computation: 42.0%
Hartree-Fock Computation: 45.0%
Hartree-Fock Computation: 48.0%
Hartree-Fock Computation: 51.0%
Hartree-Fock Computation: 54.0%
Hartree-Fock Computation: 57.0%
Hartree-Fock Computation: 60.0%
Hartree-Fock Computation: 62.0%
Hartree-Fock Computation: 65.0%
Hartree-Fock Computation: 68.0%
Hartree-Fock Computation: 71.0%
Hartree-Fock Computation: 74.0%
Hartree-Fock Computation: 77.0%
Hartree-Fock Computation: 80.0%
Hartree-Fock Computation: 82.0%
Hartree-Fock Computation: 85.0%
Hartree-Fock Computation: 88.0%
Hartree-Fock Computation: 91.0%
Hartree-Fock Computation: 94.0%
Hartree-Fock Computation: 97.0%
Hartree-

In [2]:
#Setup system for graphing purposes
import math
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

#Prepare data for graphing
morse = morsePotential(R, E)
minEIndex = E.index(min(E))
minE = E.pop(minEIndex)
optimalR = R.copy().pop(minEIndex)

In [3]:
morseData = morse.computePotential()

u = 0.5
w = (morse.a / (2 * math.pi)) * pow((2 *  morse.D / u), 0.5)
print(w)

k = 4 * math.pi * math.sqrt(2 * morse.D) / morse.a
print("K: " + str(k))

ve = []
for n in range(18):
    #ve.append( w * ((n + .5) - pow((n+.5),2)) )
    ve.append( -morse.D + (w * (n + .5)) - ( (pow(w,2) /  (4*morse.D)) * pow((n + .5), 2) )  )
print(ve)

#prepare data for use with graphing
figure = { 
    "data": [
        
        #create the Hartree-Fock generated Potential Energy Surface
        {
            "type":"scatter",
            "x":R,
            "y":E,
            "connectgaps":False,
            "mode":"markers", 
            "name":"Hartree-Fock Computed",
            "marker":{"color":"blue"}
        },
    
        #Highlight the minimum energy point in red
        {
            "type":"scatter",
            "x":[optimalR],
            "y":[minE],
            "name":"Optimal Bond Distance",
            "marker":{"color":"red"}
        },
        
        #Create and plot the Morse Potential approximation
        {
            "type":"scatter",
            "x":R,
            "y":morseData,
            "connectgaps":True,
            "name":"Morse Potential Approximation",
            "marker":{"color":"green"}
        },
        
    ],
    
    #Set up the layout of the graph
    "layout":
        {
           "xaxis":{"title":"Bond Distance in Atomic Units"},
           "yaxis":{"title":"Energy in Hartrees"},
            "title":{"text":"Hartree-Fock Energy VS Bond Distance"}
        },    
}

#Loop over all MO Energies
for vibrationalEnergy in ve:
        figure["data"].append(
            {
                "type":"scatter",
                "x":[1.1, 1.9],
                "y":[vibrationalEnergy] * 2
            }
        )
    
iplot(figure)

#print optimized bond distance
print("Optimal Bond Distance: " + str(R[minEIndex]))



Morse Potential Computation: 20%
Morse Potential Computation: 40%
Morse Potential Computation: 60%
Morse Potential Computation: 80%
Morse Potential Computation: 100%
Morse Potential Computation Complete

0.17233045610422176
K: 36.91467271140026
[-1.0400541857501813, -0.8809278054419379, -0.7350055009296729, -0.6022872722133863, -0.48277311929307787, -0.37646304216874804, -0.2833570408403964, -0.20345511530802318, -0.13675726557162854, -0.08326349163121205, -0.04297379348677399, -0.0158881711383142, -0.0020066245858327836, -0.0013291538293300764, -0.013855758868805523, -0.039586439704259124, -0.07852119633569132]


Optimal Bond Distance: 1.4000000000000001
